# Prepare User Assigned Identity (__UAI__) account for Azure AI Content Safety (__AACS__) enabled text-to-image model deployment
  

### 1. Prerequisites

#### 1.1 Assign variables

In [ ]:
uai_name = f"aacs-uai"

# The name of the AACS created in safe-text-to-image-[online/ batch]-deployments notebook
# Leaving it blank will assign all AACS resource under resources group to the UAI
aacs_name = ""

#### 1.2 Get credential

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

#### 1.3 Configure workspace 

In [ ]:
from azure.ai.ml import MLClient

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

    # get a handle to the workspace
    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)


subscription_id = ml_client.subscription_id
resource_group = ml_client.resource_group_name
workspace_name = ml_client.workspace_name
workspace_resource = ml_client.workspaces.get(workspace_name)
workspace_location = workspace_resource.location

print(f"Connected to workspace {workspace_name}")

### 2. Create a Managed Identity for the Azure AI Content Safety (AACS) enabled text-to-image endpoint

##### 2.1 Get a handle to the ManagedServiceIdentityClient

In [ ]:
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id,
    credential=credential,
)

##### 2.2 Create the User Assigned Identity:

In [ ]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters=Identity(location=workspace_location),
)

##### 2.3 Retrieve the identity object
We need to retrieve the identity object so that it can be used to deploy the Azure AI Content Safety enabled text-to-image model to an online endpoint.

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group,
    resource_name=uai_name,
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id
uai_id = uai_identity.id
print(f"UAI principal id: {uai_principal_id}")
print(f"UAI id: {uai_id}")

#### 2.4 Grant appropriate roles to the UAI we created above.
Note: In order to successfully run scripts in current step, you must have owner permission on the Azure AI Content Safety (AACS) resource and the text-to-image endpoint, which we created in the previous steps.

##### 2.4.1 Get an AuthorizationManagementClient to list Role Definitions

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.v2022_04_01.models import (
    RoleAssignmentCreateParameters,
)
from azure.core.exceptions import ResourceExistsError
import uuid

role_definition_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2022-04-01",
)
role_assignment_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2022-04-01",
)

uai_role_check_list = {
    "Cognitive Services Contributor": {
        "step": "4.2.2",
        "description": "assign Cognitive Services User role to UAI on the Azure AI Content Safety resource.",
    },
    "AcrPull": {
        "step": "4.2.3",
        "description": "assign AcrPull role to UAI on the Azure Container Registry.",
    },
    "Storage Blob Data Reader": {
        "step": "4.2.4",
        "description": "assign Storage Blob Data Reader role to UAI on the Azure Storage account.",
    },
}

##### 2.4.2 Define util function to assign access for UAI
Cognitive Services User role is required to access the Azure AI Content Safety (AACS) resource.

In [ ]:
role_name = "Cognitive Services Contributor"  # minimum role required for listing AACS access keys


def assign_access_to_aacs(scope):
    role_defs = role_definition_client.role_definitions.list(scope=scope)
    role_def = next((r for r in role_defs if r.role_name == role_name))

    try:
        role_assignment_client.role_assignments.create(
            scope=scope,
            role_assignment_name=str(uuid.uuid4()),
            parameters=RoleAssignmentCreateParameters(
                role_definition_id=role_def.id,
                principal_id=uai_principal_id,
                principal_type="ServicePrincipal",
            ),
        )
    except ResourceExistsError as ex:
        pass
    except Exception as ex:
        print(ex)
        raise ex

    if role_name in uai_role_check_list:
        del uai_role_check_list[role_name]
    print(
        f"Role assignment for {role_name} at the Azure AI Content Safety resource level completed."
    )

##### 2.4.3 Grant the user identity access to the Azure AI Content Safety (AACS) resource

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

aacs_client = CognitiveServicesManagementClient(credential, subscription_id)


def find_acs(accounts):
    return next(x for x in accounts if x.kind == "ContentSafety")


if aacs_name == "":
    for aacs_resource in aacs_client.accounts.list_by_resource_group(resource_group):
        assign_access_to_aacs(aacs_resource.id)
else:
    aacs_resource = aacs_client.accounts.get(resource_group, aacs_name)
    assign_access_to_aacs(aacs_resource.id)

##### 2.4.4 Assign AcrPull at the workspace container registry scope
Since we will create the Azure AI Content Safety (AACS) enabled text-to-image endpoint with User Assigned Identity, the user managed identity must have Storage Blob Data Reader permission on the storage account for the workspace, and AcrPull permission on the Azure Container Registry (ACR) for the workspace. Make sure your User Assigned Identity has the right permission.

In [ ]:
container_registry = workspace_resource.container_registry

role_name = "AcrPull"

role_defs = role_definition_client.role_definitions.list(scope=container_registry)
role_def = next((r for r in role_defs if r.role_name == role_name))

try:
    role_assignment_client.role_assignments.create(
        scope=container_registry,
        role_assignment_name=str(uuid.uuid4()),
        parameters=RoleAssignmentCreateParameters(
            role_definition_id=role_def.id,
            principal_id=uai_principal_id,
            principal_type="ServicePrincipal",
        ),
    )
except ResourceExistsError as ex:
    pass
except Exception as ex:
    print(ex)
    raise ex

if role_name in uai_role_check_list:
    del uai_role_check_list[role_name]
print("Role assignment for AcrPull at the workspace container registry completed.")

##### 2.4.6 Assign Storage Blob Data Reader at the workspace storage account scope

In [ ]:
role_name = "Storage Blob Data Reader"
blob_scope = workspace_resource.storage_account

role_defs = role_definition_client.role_definitions.list(scope=blob_scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

try:
    role_assignment_client.role_assignments.create(
        scope=blob_scope,
        role_assignment_name=str(uuid.uuid4()),
        parameters=RoleAssignmentCreateParameters(
            role_definition_id=role_def.id,
            principal_id=uai_principal_id,
            principal_type="ServicePrincipal",
        ),
    )
except ResourceExistsError as ex:
    pass
except Exception as ex:
    print(ex)
    raise ex

if role_name in uai_role_check_list:
    del uai_role_check_list[role_name]
print(
    "Role assignment for `Storage Blob Data Reader` at the workspace storage account completed."
)

2.4.7 Let's make sure we didn't miss anything in the previous steps, please execute the following script to check on that:

In [ ]:
# Check everything is properly done before creating the Azure AI Content Safety enabled text-to-image online endpoint
missing_steps = []
print(
    "You have selected UAI to deploy the Azure AI Content Safety Enabled text-to-image online endpoint, checking if the UAI has the required roles assigned..."
)
if uai_role_check_list:
    for key, value in uai_role_check_list.items():
        missing_steps.append(
            f'Please go to step {value["step"]} to {value["description"]}'
        )

if missing_steps:
    print("Seems you missed some step above.")
    steps = "\n".join(missing_steps)
    raise Exception(f"Please complete the missing steps before proceeding:\n{steps}")
else:
    print(
        "All steps are completed, proceeding to create the Azure AI Content Safety Enabled text-to-image online endpoint...\n"
        f"uai_id: {uai_id}\n"
        f"uai_client_id: {uai_client_id}"
    )